# Generating Features from GeoTiff Files
From GeoTiff Files available for India over a period of more than 20 years, we want to generate features from those files for the problem of prediction of district wise crop yield in India.

Due to gdal package, had to make a separate environment using conda. So install packages for this notebook in that environment itself. Check from the anaconda prompt, the names of all the envs are available: $conda info --envs 

In [5]:
from osgeo import ogr, osr, gdal

import requests # To make the REST API call
import json

import numpy as np
import pandas as pd